# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1ea0f7ae80>
├── 'a' --> tensor([[ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334]])
└── 'x' --> <FastTreeValue 0x7f1ea0f7afd0>
    └── 'c' --> tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                        [ 0.0362, -0.3067,  0.6085, -1.8128],
                        [-0.6217,  2.0139, -0.4573, -0.0803]])

In [4]:
t.a

tensor([[ 2.0097,  0.4762, -1.5782],
        [-0.3751,  0.0518, -0.4334]])

In [5]:
%timeit t.a

62.4 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1ea0f7ae80>
├── 'a' --> tensor([[-0.2275, -0.0053, -0.4883],
│                   [-1.8014, -1.3524, -0.4291]])
└── 'x' --> <FastTreeValue 0x7f1ea0f7afd0>
    └── 'c' --> tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                        [ 0.0362, -0.3067,  0.6085, -1.8128],
                        [-0.6217,  2.0139, -0.4573, -0.0803]])

In [7]:
%timeit t.a = new_value

70.4 ns ± 0.0554 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.0097,  0.4762, -1.5782],
               [-0.3751,  0.0518, -0.4334]]),
    x: Batch(
           c: tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803]]),
       ),
)

In [10]:
b.a

tensor([[ 2.0097,  0.4762, -1.5782],
        [-0.3751,  0.0518, -0.4334]])

In [11]:
%timeit b.a

58.3 ns ± 0.0449 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0941,  1.4889,  1.0317],
               [-0.2321, -0.0324, -1.3299]]),
    x: Batch(
           c: tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803]]),
       ),
)

In [13]:
%timeit b.a = new_value

514 ns ± 0.459 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.343 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 640 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 237 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1ea0f7ad90>
├── 'a' --> tensor([[[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]],
│           
│                   [[ 2.0097,  0.4762, -1.5782],
│                    [-0.3751,  0.0518, -0.4334]]])
└── 'x' --> <FastTreeValue 0x7f1ea0f7af40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1e783e1a60>
├── 'a' --> tensor([[ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334],
│                   [ 2.0097,  0.4762, -1.5782],
│                   [-0.3751,  0.0518, -0.4334]])
└── 'x' --> <FastTreeValue 0x7f1df5318df0>
    └── 'c' --> tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                        [ 0.0362, -0.3067,  0.6085, -1.8128],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 74.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.4112, -1.0521,  0.5175,  2.3693],
                       [ 0.0362, -0.3067,  0.6085, -1.8128],
                       [-0.6217,  2.0139, -0.4573, -0.0803]],
              
                      [[ 1.4112, -1.0521,  0.5175,  2.3693],
                       [ 0.0362, -0.3067,  0.6085, -1.8128],
                       [-0.6217,  2.0139, -0.4573, -0.0803]],
              
                      [[ 1.4112, -1.0521,  0.5175,  2.3693],
                       [ 0.0362, -0.3067,  0.6085, -1.8128],
                       [-0.6217,  2.0139, -0.4573, -0.0803]],
              
                      [[ 1.4112, -1.0521,  0.5175,  2.3693],
                       [ 0.0362, -0.3067,  0.6085, -1.8128],
                       [-0.6217,  2.0139, -0.4573, -0.0803]],
              
                      [[ 1.4112, -1.0521,  0.5175,  2.3693],
                       [ 0.0362, -0.3067,  0.6085, -1.8128],
                       [-0.6217,  2.0139, -0.4573, -0.0803]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803],
                      [ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803],
                      [ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803],
                      [ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803],
                      [ 1.4112, -1.0521,  0.5175,  2.3693],
                      [ 0.0362, -0.3067,  0.6085, -1.8128],
                      [-0.6217,  2.0139, -0.4573, -0.0803],
                      [ 1.4112, -1.0521,  0.5175,  2.3693],
                   

In [28]:
%timeit Batch.cat(batches)

155 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

334 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
